In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import get_module_performance

pd.set_option("display.precision", 2)

In [3]:
device = 'cuda:0'
device

'cuda:0'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device, 
                                                               track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.47,0.04,0.61,0.03,32994.5,128.0,545259520.0,33280.0,1419.19
conv_64_512_3x3,0.84,0.02,1.35,0.03,35972.0,1152.0,4690151424.0,295424.0,764.46
bottleneck_64_512_3x3,0.88,0.3,1.58,0.03,36420.0,2624.0,2383218688.0,150048.0,848.36
conv_64_512_3x3_g2,1.96,0.02,2.74,0.03,34032.5,3136.0,11956733952.0,803328.0,1624.74
conv_64_512_3x3_g8,0.68,0.01,1.54,0.03,32210.0,784.0,2994898944.0,201216.0,778.49
conv_padding,0.78,0.01,1.04,0.47,35972.0,89216.0,4840226816.0,295424.0,634.78
conv_nn_Padding,0.84,0.01,1.33,0.03,40324.0,1152.0,4840226816.0,295424.0,759.17


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.46,0.02,0.47,0.09,36.47,34.0,545259520.0,33280.0,331.26
conv_64_512_3x3,1.0,0.01,1.39,0.03,37.22,34.0,4690151424.0,295424.0,826.0
bottleneck_64_512_3x3,0.92,0.35,2.46,0.04,39.65,34.0,2383218688.0,150048.0,1158.36
conv_64_512_3x3_g2,1.9,0.01,13.61,0.05,37.42,39.2,11956733952.0,803328.0,5301.06
conv_64_512_3x3_g8,1.07,0.02,12.23,0.05,35.64,34.77,2994898944.0,201216.0,4546.94
conv_padding,0.98,0.01,2.05,0.04,38.19,34.0,4840226816.0,295424.0,1045.68
conv_nn_Padding,1.06,0.01,1.38,0.03,42.25,34.0,4840226816.0,295424.0,845.85


In [7]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.4,0.01,0.36,0.01,0.04,0.03,545259520.0,33280.0,289.97
conv_64_512_3x3,0.53,0.02,0.64,0.07,0.04,0.03,4690151424.0,295424.0,414.84
bottleneck_64_512_3x3,0.82,0.05,1.21,0.08,0.04,0.03,2383218688.0,150048.0,743.94
conv_64_512_3x3_g2,10.66,0.02,12.11,0.04,0.04,0.04,11956733952.0,803328.0,7772.07
conv_64_512_3x3_g8,2.55,0.02,10.14,0.03,0.03,0.03,2994898944.0,201216.0,4343.91
conv_padding,0.51,0.02,0.53,0.09,0.04,0.03,4840226816.0,295424.0,375.37
conv_nn_Padding,0.58,0.01,0.63,0.09,0.04,0.03,4840226816.0,295424.0,430.83


In [8]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.4,0.0,0.37,0.04,37980672.0,35654144.0,545259520.0,33280.0,294.39
conv_64_512_3x3,0.52,0.01,0.53,0.09,38805504.0,35654144.0,4690151424.0,295424.0,374.09
bottleneck_64_512_3x3,0.81,0.05,1.2,0.08,39493120.0,35654144.0,2383218688.0,150048.0,732.78
conv_64_512_3x3_g2,10.66,0.02,12.12,0.02,38068736.0,41105920.0,11956733952.0,803328.0,7773.11
conv_64_512_3x3_g8,2.55,0.01,10.14,0.03,36202496.0,36454400.0,2994898944.0,201216.0,4345.67
conv_padding,0.52,0.02,0.6,0.19,39845888.0,35654144.0,4840226816.0,295424.0,396.75
conv_nn_Padding,0.59,0.02,0.67,0.24,44304384.0,35654144.0,4840226816.0,295424.0,445.76
